# 宏观经济特征提取

- <= yc_macro_econ
- 输出各种专用数据表以及统计表
- 并尝试针对性增补

## 背景

> Season2/yc_macro_econ4car.csv 

直觉选定24则宏观指数

In [1]:
eco_indicator = (
("ycmecon_CPIPPI_all_industrial_m2m" ,"CPI与PPI","PPI:全部工业品:当月同比(%)")
,("ycmecon_CPIPPI_all_industrial_lrratio" ,"CPI与PPI","PPI:全部工业品:环比(%)")
,("ycmecon_PMI_Produce" ,"PMI","PMI:生产(%)")
,("ycmecon_PMI_Purchases" ,"PMI","PMI:采购量(%)")
,("ycmecon_PMI_inventory_raw" ,"PMI","PMI:原材料库存(%)")
,("ycmecon_PMI_Consumer_ConfIndex_mon" ,"PMI","消费者信心指数:月")
,("ycmecon_PMI_Consumer_ExpIndex_mon" ,"PMI","消费者预期指数:月")
,("ycmecon_order_index_PMI" ,"订单指数","汇丰PMI:新订单(%)")
,("ycmecon_order_index_LGSC_PMI" ,"订单指数","兰格钢铁:钢铁流通业采购经理人指数(LGSC-PMI):总订单量(%)")
,("ycmecon_invest_fixed_infrastructure_y4y" ,"分行业固定资产投资","固定资产投资完成额:基础设施建设投资:累计同比(%)")
,("ycmecon_invest_fixed_automotive_y4y" ,"分行业固定资产投资","固定资产投资完成额:制造业:汽车制造业:累计同比(%)")
,("ycmecon_industrial_output_steel_m2m" ,"工业品","产量:钢材:当月同比(%)")
,("ycmecon_industrial_output_oreiron_m2m" ,"工业品","产量:铁矿石原矿量:当月同比(%)")
,("ycmecon_industrial_added_m2m" ,"工业增加值","工业增加值:当月同比(%)")
,("ycmecon_industrial_added_y4y" ,"工业增加值","工业增加值:累计同比(%)")
,("ycmecon_inventory_LGSC_PMI" ,"库存指数","兰格钢铁:钢铁流通业采购经理人指数(LGSC-PMI):库存水平(%)")
,("ycmecon_in_rate_current_deposit_mon" ,"利率汇率","活期存款利率(月)(%)")
,("ycmecon_ex_rate_USD2RMB" ,"利率汇率","平均汇率:美元兑人民币")
,("ycmecon_ex_rate_EUP2RMB" ,"利率汇率","平均汇率:欧元兑人民币")
,("ycmecon_ex_rate_HKD2RMB" ,"利率汇率","平均汇率:港元兑人民币")
,("ycmecon_ex_rate_JPY2RMB" ,"利率汇率","平均汇率:100日元兑人民币")
,("ycmecon_retail_goods_m2m" ,"社会消费品零售总额","社会消费品零售总额:当月同比(%)")
,("ycmecon_retail_oil_m2m" ,"社会消费品零售总额","零售额:石油及制品类:当月同比(%)")
,("ycmecon_retail_car_monthly" ,"社会消费品零售总额","零售额:汽车类:当月同比(%)")
)

In [2]:
for eco in eco_indicator:
    tname,_type,_indi = eco
    print('{}-{}-{}'.format(tname,_type,_indi))

ycmecon_CPIPPI_all_industrial_m2m-CPI与PPI-PPI:全部工业品:当月同比(%)
ycmecon_CPIPPI_all_industrial_lrratio-CPI与PPI-PPI:全部工业品:环比(%)
ycmecon_PMI_Produce-PMI-PMI:生产(%)
ycmecon_PMI_Purchases-PMI-PMI:采购量(%)
ycmecon_PMI_inventory_raw-PMI-PMI:原材料库存(%)
ycmecon_PMI_Consumer_ConfIndex_mon-PMI-消费者信心指数:月
ycmecon_PMI_Consumer_ExpIndex_mon-PMI-消费者预期指数:月
ycmecon_order_index_PMI-订单指数-汇丰PMI:新订单(%)
ycmecon_order_index_LGSC_PMI-订单指数-兰格钢铁:钢铁流通业采购经理人指数(LGSC-PMI):总订单量(%)
ycmecon_invest_fixed_infrastructure_y4y-分行业固定资产投资-固定资产投资完成额:基础设施建设投资:累计同比(%)
ycmecon_invest_fixed_automotive_y4y-分行业固定资产投资-固定资产投资完成额:制造业:汽车制造业:累计同比(%)
ycmecon_industrial_output_steel_m2m-工业品-产量:钢材:当月同比(%)
ycmecon_industrial_output_oreiron_m2m-工业品-产量:铁矿石原矿量:当月同比(%)
ycmecon_industrial_added_m2m-工业增加值-工业增加值:当月同比(%)
ycmecon_industrial_added_y4y-工业增加值-工业增加值:累计同比(%)
ycmecon_inventory_LGSC_PMI-库存指数-兰格钢铁:钢铁流通业采购经理人指数(LGSC-PMI):库存水平(%)
ycmecon_in_rate_current_deposit_mon-利率汇率-活期存款利率(月)(%)
ycmecon_ex_rate_USD2RMB-利率汇率-平均汇率:美元兑人民币
ycmecon_ex_rate_EUP2RMB-利率汇率-

In [3]:
tpl_SQL = '''
-- {mc_tname}_{mc_type}_{mc_indi}
-- 提取合理宏观特征为数值表
-- <= yc_macro_econ
-- => {mc_tname}_grow
-- => {mc_tname}_grow_avg
-- 
-- 
DROP TABLE IF EXISTS {mc_tname}_grow;
-- 
-- 
CREATE TABLE IF NOT EXISTS {mc_tname}_grow
AS
SELECT date, substring(date, 1, 6) AS month
	, number
FROM yc_macro_econ
WHERE eco_indicator_type = '{mc_type}'
	AND eco_indicator = '{mc_indi}'
	AND number IS NOT NULL
DISTRIBUTE BY date SORT BY date;
-- 
-- 
DROP TABLE IF EXISTS {mc_tname}_grow_avg;
-- 
-- 
CREATE TABLE IF NOT EXISTS {mc_tname}_grow_avg
AS
SELECT month, AVG(number) AS avg
FROM {mc_tname}_grow
GROUP BY month;
-- 
-- 
SELECT *
FROM {mc_tname}_grow_avg
LIMIT 100;
'''


In [5]:
_p2sql = '../../Season2/ycmecon_sql/{}.sql'
for eco in eco_indicator:
    tname,_type,_indi = eco
    #print('{}-{}-{}'.format(tname,_type,_indi))
    _stuff = {'mc_tname': tname
              , 'mc_type': _type
              , 'mc_indi': _indi
             }
    #print(tpl_SQL.format(**_stuff))
    _sql = _p2sql.format(tname)
    print('嗯哼为:%s'% _sql)
    open(_sql,'w').write(tpl_SQL.format(**_stuff))

嗯哼为:../../Season2/ycmecon_CPIPPI_all_industrial_m2m.sql
嗯哼为:../../Season2/ycmecon_CPIPPI_all_industrial_lrratio.sql
嗯哼为:../../Season2/ycmecon_PMI_Produce.sql
嗯哼为:../../Season2/ycmecon_PMI_Purchases.sql
嗯哼为:../../Season2/ycmecon_PMI_inventory_raw.sql
嗯哼为:../../Season2/ycmecon_PMI_Consumer_ConfIndex_mon.sql
嗯哼为:../../Season2/ycmecon_PMI_Consumer_ExpIndex_mon.sql
嗯哼为:../../Season2/ycmecon_order_index_PMI.sql
嗯哼为:../../Season2/ycmecon_order_index_LGSC_PMI.sql
嗯哼为:../../Season2/ycmecon_invest_fixed_infrastructure_y4y.sql
嗯哼为:../../Season2/ycmecon_invest_fixed_automotive_y4y.sql
嗯哼为:../../Season2/ycmecon_industrial_output_steel_m2m.sql
嗯哼为:../../Season2/ycmecon_industrial_output_oreiron_m2m.sql
嗯哼为:../../Season2/ycmecon_industrial_added_m2m.sql
嗯哼为:../../Season2/ycmecon_industrial_added_y4y.sql
嗯哼为:../../Season2/ycmecon_inventory_LGSC_PMI.sql
嗯哼为:../../Season2/ycmecon_in_rate_current_deposit_mon.sql
嗯哼为:../../Season2/ycmecon_ex_rate_USD2RMB.sql
嗯哼为:../../Season2/ycmecon_ex_rate_EUP2RMB.sql
嗯